In [1]:
#Khai báo các thư viện cần thiết
import os
import re
from collections import Counter
import base64

import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

IMG_SIZE = 256
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model = load_model('best_model_256.h5')


In [2]:
#Link tới trang web mua bán ô tô ở chợ tốt xét theo các page
website='https://xe.chotot.com/mua-ban-oto?page='

In [3]:
href_list = []
partner_list = []
# Chạy vòng lặp để gắn các số vào trong đường link web site -> thu thập dữ liệu mỗi link
for i in range(1, 250):
    # Gắn số vào link
    url = website + str(i)
    print(url)
    # Tạo một request đến url đó để truy cập web
    response = requests.get(url)

    # Kiểm tra truy cập thành công hay không
    if response.status_code == 200:
        # Tách phần html format và đưa vào lưu trữ trong biến soup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Dò web site thì thấy các tin thường được đăng dưới tag <li> với class là AdItem_wrapperAdItem__S6qPH
        li_tags = soup.find_all('li', class_="AdItem_wrapperAdItem__S6qPH")

        # Chạy để lấy thông tin từng thẻ <li>
        for li_tag in li_tags:
            # Các đường dẫn vào chi tiết của chiếc xe sẽ nằm trong thẻ a với class sau
            a_tag = li_tag.find('a', class_="AdItem_adItem__gDDQT")

            # Rút trích đường link dẫn tới page chi tiết xe
            href = a_tag.get('href') if a_tag else None
            href_list.append(href)

            # Tìm kiếm thẻ đối tác trong từng thẻ <li>
            div_protected_label = li_tag.find('div', class_="AdBody_protectedLabelInTitle__HQgj6")

            # Nếu có là đối tác thì điền "Yes" còn không thì "No"
            partner = "Yes" if div_protected_label else "No"
            partner_list.append(partner)
    else:
        print(f"Failed to retrieve the webpage: {url}")

    

https://xe.chotot.com/mua-ban-oto?page=1
https://xe.chotot.com/mua-ban-oto?page=2
https://xe.chotot.com/mua-ban-oto?page=3
https://xe.chotot.com/mua-ban-oto?page=4
https://xe.chotot.com/mua-ban-oto?page=5
https://xe.chotot.com/mua-ban-oto?page=6
https://xe.chotot.com/mua-ban-oto?page=7
https://xe.chotot.com/mua-ban-oto?page=8
https://xe.chotot.com/mua-ban-oto?page=9
https://xe.chotot.com/mua-ban-oto?page=10
https://xe.chotot.com/mua-ban-oto?page=11
https://xe.chotot.com/mua-ban-oto?page=12
https://xe.chotot.com/mua-ban-oto?page=13
https://xe.chotot.com/mua-ban-oto?page=14
https://xe.chotot.com/mua-ban-oto?page=15
https://xe.chotot.com/mua-ban-oto?page=16
https://xe.chotot.com/mua-ban-oto?page=17
https://xe.chotot.com/mua-ban-oto?page=18
https://xe.chotot.com/mua-ban-oto?page=19
https://xe.chotot.com/mua-ban-oto?page=20
https://xe.chotot.com/mua-ban-oto?page=21
https://xe.chotot.com/mua-ban-oto?page=22
https://xe.chotot.com/mua-ban-oto?page=23
https://xe.chotot.com/mua-ban-oto?page=24
h

In [4]:
# Tạo dataframe để lưu trữ href và Đối tác
df = pd.DataFrame({'href': href_list, 'Đối tác': partner_list})

In [5]:
#Lưu trữ dataframe
df.to_csv('CrawlLink.csv', index=False)

In [6]:
print(df)

                                                   href Đối tác
0     /mua-ban-oto-quan-go-vap-tp-ho-chi-minh/109479...      No
1     /mua-ban-oto-quan-go-vap-tp-ho-chi-minh/111954...      No
2     /mua-ban-oto-quan-go-vap-tp-ho-chi-minh/111554...     Yes
3     /mua-ban-oto-quan-go-vap-tp-ho-chi-minh/110916...     Yes
4     /mua-ban-oto-quan-phu-nhuan-tp-ho-chi-minh/111...      No
...                                                 ...     ...
4985  /mua-ban-oto-quan-thanh-xuan-ha-noi/111739791.htm     Yes
4986  /mua-ban-oto-quan-binh-thanh-tp-ho-chi-minh/11...      No
4987  /mua-ban-oto-quan-bac-tu-liem-ha-noi/111968082...      No
4988  /mua-ban-oto-quan-binh-thanh-tp-ho-chi-minh/11...      No
4989   /mua-ban-oto-quan-7-tp-ho-chi-minh/111968046.htm      No

[4990 rows x 2 columns]


In [7]:
# Sau khi thu thập các href thì ấn vô ko truy cập được do thiếu nguồn của đường link nên ta sẽ thêm vô đầu mỗi href
df = pd.read_csv('CrawlLink.csv')
def add_base_url(href):
    return 'https://xe.chotot.com' + href

# Apply the function to the "href" column to add the base URL
df['href'] = df['href'].apply(add_base_url)
df.to_csv('FullLink.csv', index=False)

In [8]:
df['href'][0]

'https://xe.chotot.com/mua-ban-oto-quan-go-vap-tp-ho-chi-minh/109479662.htm#px=SR-stickyad-[PO-1][PL-top]'

In [9]:
df= pd.read_csv('./FullLink.csv')

In [10]:
df

,href,Đối tác
0,https://xe.chotot.com/mua-ban-oto-quan-go-vap-...,No
1,https://xe.chotot.com/mua-ban-oto-quan-go-vap-...,No
2,https://xe.chotot.com/mua-ban-oto-quan-go-vap-...,Yes
3,https://xe.chotot.com/mua-ban-oto-quan-go-vap-...,Yes
4,https://xe.chotot.com/mua-ban-oto-quan-phu-nhu...,No
...,...,...
4985,https://xe.chotot.com/mua-ban-oto-quan-thanh-x...,Yes
4986,https://xe.chotot.com/mua-ban-oto-quan-binh-th...,No
4987,https://xe.chotot.com/mua-ban-oto-quan-bac-tu-...,No
4988,https://xe.chotot.com/mua-ban-oto-quan-binh-th...,No


In [11]:
column_names = [
    'Màu sắc',
    'Tiêu đề',
    'Chủ',
    'Phân loại người bán',
    'Địa chỉ',
    'Hãng',
    'Dòng xe',
    'Năm sản xuất',
    'Số Km đã đi',
    'Tình trạng',
    'Hộp số',
    'Nhiên liệu',
    'Kiểu dáng',
    'Số chỗ',
    'Xuất xứ',
    'Chính sách bảo hành',
    'Trọng lượng',
    'Trọng tải',
    'Giá'
    ]

In [12]:
def extract_data_with_a_tag(soup, itemprop):
    element = soup.find('a', itemprop=itemprop)
    return element.get_text(strip=True) if element else '-'

def extract_data(soup, itemprop):
    element = soup.find('span', itemprop=itemprop)    
    return element.get_text(strip=True) if element else '-'
    
def click_button(driver, class_name):
    button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
    button.click()

In [13]:
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = image.img_to_array(img)
    img = img / 255.0  # Rescale to match the data generators
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

In [14]:
class_to_color = {
    0: 'be',
    1: 'đen',
    2: 'xanh_dương',
    3: 'nâu',
    4: 'gold',
    5: 'xanh_lá',
    6: 'xám',
    7: 'cam',
    8: 'hồng',
    9: 'tím',
    10: 'đỏ',
    11: 'trắng',
    12: 'vàng'
}

In [15]:
df['href'][0]

'https://xe.chotot.com/mua-ban-oto-quan-go-vap-tp-ho-chi-minh/109479662.htm#px=SR-stickyad-[PO-1][PL-top]'

In [16]:
dfs =[]
count=0

for i in range(1, 2):
    print("working on url: ", i)
    driver = webdriver.Firefox()
    url = df['href'][i]
    driver.get(df['href'][i])
    count += 1
    try:
        # Chờ và nhấn nút để đóng popup
        click_button(driver, "aw__s13upwxx")
        click_button(driver, "styles_button__SVZnw")

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        location_element = soup.find('span', class_='fz13') 
        location_text = location_element.get_text(strip=True) if location_element else '-'

        owner_element = soup.find('b', role='presentation') 
        owner_text = owner_element.get_text(strip=True) if owner_element else '-'

        seller_element = soup.find('div', class_='SellerProfile_statusOnlineDivVeh__kgmx4') 
        seller_text = seller_element.get_text(strip=True) if seller_element else '-'

        frist_div = soup.find('div', class_='DetailViewAB_adviewItem__20YCA')
        if frist_div:
            # Tìm thẻ span với thuộc tính itemprop="price" bên trong div_element
            name_element = frist_div.find('h1', itemprop='name', class_='AdDecriptionVeh_adTitle__vEuKD')
            name_text = name_element.get_text(strip=True) if name_element else '-'

            
            flag = 1
            tag_content = soup.find('p', class_='styles_adBody__vGW74').get_text(strip=True)
            cleaned_content = re.sub(r'[^\w\s]', '', tag_content)  # Loại bỏ kí tự đặc biệt
            word_list = cleaned_content.split()
            for i, word in enumerate(word_list):
                if word.lower() == 'màu':
                    if i + 1 < len(word_list):
                        # Kiểm tra nếu từ kế tiếp là 'sắc' hoặc 'Sắc'
                        if word_list[i + 1].lower() == 'sắc':
                            color_text.append(word_list[i + 2])
                        else:
                            color_text.append(word_list[i + 1])
                        flag = 0
                        break
            if flag == 1:
                img_elements = soup.find_all("img", {"alt": name_text})
                predicted_classes = []
                for i in range(0, len(img_elements)-1):
                    img_url = img_elements[i]["src"]
                    # Check if img_url is a base64-encoded image
                    if not img_url.startswith("data:image"):
                        img_data = requests.get(img_url).content
                        with open(f'data/ten_anh_{i}.jpg', 'wb') as img_file:
                            img_file.write(img_data)
                        
                        image_path = f'./data/ten_anh_{i}.jpg'
                    else:
                        
                        with open(image_path, 'wb') as img_file:
                            img_file.write(base64.b64decode(img_url.split(",")[1]))
                    # Preprocess the image and make predictions
                    preprocessed_image = preprocess_image(image_path)
                    predictions = model.predict(preprocessed_image)

                    # Get the predicted class index for the current image
                    predicted_class_index = np.argmax(predictions, axis=1)
                    predicted_classes.append(predicted_class_index[0])

                # Count the occurrences of each predicted class
                class_counts = Counter(predicted_classes)

                # Find the most common predicted class
                most_common_class = class_counts.most_common(1)[0]

                # The 'most_common_class' variable will contain a tuple (class, count).
                # You can access the class as most_common_class[0] and the count as most_common_class[1].
                most_common_class_index = most_common_class[0]
                color_text = class_to_color[most_common_class_index]

            price_element = frist_div.find('span', itemprop='price')
            price_text = price_element.get_text(strip=True) if price_element else '-'

        else:
            print("Không tìm thấy div với class 'DetailViewAB_adviewItem__20YCA'.")
        
        div_element = soup.find('div', class_='AdParam_adParamContainerVeh__Vz4Zt')
        if div_element:
            carbrand_text = extract_data_with_a_tag(div_element, 'carbrand')
            carmodel_text = extract_data_with_a_tag(div_element, 'carmodel')
            Prodate_text = extract_data(div_element, 'mfdate')
            KM_text = extract_data(div_element, 'mileage_v2')
            Condition_text = extract_data(div_element, 'condition_ad')
            Gear_text = extract_data(div_element, 'gearbox')
            Fuel_text = extract_data(div_element, 'fuel')
            cartype_text = extract_data(div_element, 'cartype')
            carseats_text = extract_data(div_element, 'carseats')
            carorigin_text = extract_data(div_element, 'carorigin')
            veh_warranty_policy_text= extract_data(div_element, 'veh_warranty_policy')
            veh_unladen_weight_text= extract_data(div_element, 'veh_unladen_weight')
            veh_gross_weight_text = extract_data(div_element, 'veh_gross_weight')
            
            extract_data_list = [color_text, name_text,owner_text,seller_text,location_text,carbrand_text,carmodel_text,Prodate_text,KM_text,Condition_text,
                        Gear_text,Fuel_text,cartype_text,carseats_text,carorigin_text,
                        veh_warranty_policy_text,veh_unladen_weight_text,veh_gross_weight_text,price_text]
        else:
            print("Không thể tìm thấy div với class 'AdParam_adParamContainerVeh__Vz4Zt'.")
    except Exception as e:
        print(f"Failed to retrieve data from URL: {url}")
        print(f"Error: {str(e)}")
        extract_data_list = ['-'] * 19
    new_df = pd.DataFrame([extract_data_list], columns=column_names)
    dfs.append(new_df)
    driver.quit()
final_df = pd.concat(dfs, ignore_index=True, axis=0)

working on url:  1
1/1 [==============================] - 0s 14ms/step


In [17]:
final_df = pd.concat(dfs, ignore_index=True, axis=0)

In [18]:
final_df

,Màu sắc,Tiêu đề,Chủ,Phân loại người bán,Địa chỉ,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Kiểu dáng,Số chỗ,Xuất xứ,Chính sách bảo hành,Trọng lượng,Trọng tải,Giá
0,đen,Innova G tự động 08 chỗ 2014 vàng cát siêu keng,Phan Trung Quân,Bán chuyên,"1/156, Hẻm 156 Lê Văn Thọ, Phường 11, Quận Gò ...",Toyota,Innova,2014,78000,Đã sử dụng,Tự động,Xăng,Minivan (MPV),7,Việt Nam,Bảo hành hãng,> 1 tấn,> 2 tấn,386.000.000 đ


In [19]:

final_df['Đối tác'] = df['Đối tác']
final_df['href'] = df['href']
final_df.to_csv('./CarPricePrediction_test_3288_4859.csv', index=False)